UPDATE

I got the triplet loss class and functions but it's not working yet. The code came from this url: https://github.com/Yuol96/pytorch-triplet-loss

In [ ]:
# !wget https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
# !unzip -q dataset.zip -d dataset

--2021-06-22 19:51:18--  https://market1501.s3-us-west-2.amazonaws.com/dataset.zip
Resolving market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)... 52.218.160.81
Connecting to market1501.s3-us-west-2.amazonaws.com (market1501.s3-us-west-2.amazonaws.com)|52.218.160.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 82925180 (79M) [application/zip]
Saving to: ‘dataset.zip.2’

dataset.zip.2       100%[===================>]  79.08M  35.4MB/s    in 2.2s    

2021-06-22 19:51:20 (35.4 MB/s) - ‘dataset.zip.2’ saved [82925180/82925180]

replace dataset/annotations_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
# !rm -rf /content/deeplearning_unitn
# !git clone https://github.com/deayalar/deeplearning_unitn.git

Cloning into 'deeplearning_unitn'...
remote: Enumerating objects: 192, done.
remote: Counting objects: 100% (192/192), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 192 (delta 106), reused 119 (delta 53), pack-reused 0
Receiving objects: 100% (192/192), 5.49 MiB | 4.46 MiB/s, done.
Resolving deltas: 100% (106/106), done.


In [ ]:
!nvidia-smi

Tue Jun 22 20:26:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   68C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
config = dict(
    wandb = True,
    device = "auto", # Select an specific device None to select automatically
    train_root = "/content/dataset/train",
    test_root = "/content/dataset/test", 
    queries_root = "/content/dataset/queries",
    attributes_file = "/content/dataset/annotations_train.csv",
    #train_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/train",
    #test_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/test",
    #queries_root = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/queries",
    #attributes_file = "/media/deayalar/Data/Documents/Unitn/Deep Learning/Assignment/dataset/annotations_train.csv",
    dataset="Market1501",
    backbone = "resnet18",
    split = dict(
        full_training_size = 0.75,
        train_size = 0.8
    ),
    compose = dict(
        resize_h = 224,
        resize_w = 224
    ),
    epochs=1,
    training_batch_size=8,
    validation_batch_size=32,
    learning_rate=0.01,
    weight_decay=0.000001, 
    momentum=0.9,
    test_before_training=True,
    test_after_epochs=10,
    mAP_rank=10)

In [ ]:
%cd /content/deeplearning_unitn

import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
from tqdm.notebook import tqdm

import cost_functions
from evaluation import Evaluator
from my_datasets.reid_dataset import Market1501
from cost_functions import AttributesLossWrapper
from utils.split_data import ValidationSplitter, TrainingSplitter
from models.reid_model import FinetunedModel
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

if config["device"] == "auto":
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
else:
    device = config["device"]
print(device)

/content/deeplearning_unitn
cuda:0


In [ ]:
# !pip install wandb -q
# import wandb
# wandb.login()

     |████████████████████████████████| 1.8MB 621kB/s 
     |████████████████████████████████| 133kB 2.3MB/s 
     |████████████████████████████████| 102kB 2.4MB/s 
     |████████████████████████████████| 174kB 2.5MB/s 
     |████████████████████████████████| 71kB 2.2MB/s 


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
def model_pipeline(hyperparameters):
    """
    This function corresponds to the basic pipeline of all tested models
    0) Split data
    1) Setup based on the configuration
    2) Train the model
    3) Test performance
    """
    config = hyperparameters
    if config["wandb"]:
      wandb.init(entity="dl_unitn", project="dl_project", config=hyperparameters)
      config = wandb.config
    print(config)
    
    train_set, val_set, val_queries = split_data(config)
    
    model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
    id_ground_truth_dict = build_ground_truth(val_set, val_queries)

    print("Using "+ config["backbone"] + " as backbone")
    if config["test_before_training"]:
      test(model, val_loader, val_queries_loader, id_ground_truth_dict, config)

    train(model, train_loader, val_loader, criterion, optimizer, config)

    test(model, val_loader, val_queries_loader, id_ground_truth_dict, config, save_model=True)

    return model

In [ ]:
def build_ground_truth(val_set, val_queries):
    values = []
    for q in val_queries:
        matches = []
        for idx_v, v in enumerate(val_set):
            if v.split("_")[0] == q.split("_")[0]:
                matches.append(idx_v)
        value = set(matches)
        values.append(value)
        
    ground_truth_dict = dict(zip(list(range(0, len(val_queries))), values))
    return ground_truth_dict


In [ ]:
def split_data(config):
    """Returns a list with the names of theimages in each set"""
    splitter = ValidationSplitter(train_root=config["train_root"], 
                                  test_root=config["test_root"], 
                                  queries_root=config["queries_root"])
    train_set, val_set, val_queries = splitter.split(train_size=config["split"]["full_training_size"],
                                                     random_seed=42)
    return train_set, val_set, val_queries

def setup(train_set, val_set, val_queries, config):
    #Create pytorch Datasets 
    composed = transforms.Compose([transforms.Resize((config["compose"]["resize_h"], 
                                                      config["compose"]["resize_w"])),
                                   transforms.ToTensor(),
                                   transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                        std=[0.229, 0.224, 0.225])])
    
    train_dataset = Market1501(root_dir=config["train_root"],
                            attributes_file=config["attributes_file"],
                            #full_train_set=full_train_set,
                            images_list=train_set,
                            transform=composed)
                            
    val_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         #full_train_set = full_train_set,
                         images_list=val_set,
                         transform=composed)

    val_queries_dataset = Market1501(root_dir=config["train_root"],
                         attributes_file=config["attributes_file"],
                         #full_train_set = full_train_set,
                         images_list=val_queries,
                         transform=composed)

    train_loader = torch.utils.data.DataLoader(train_dataset, 
                                               batch_size=config["training_batch_size"], 
                                               shuffle=True, 
                                               num_workers=8)
                                               
    val_loader = torch.utils.data.DataLoader(val_dataset, 
                                             batch_size=config["validation_batch_size"], 
                                             shuffle=False, 
                                             num_workers=8)

    val_queries_loader = torch.utils.data.DataLoader(val_queries_dataset, 
                                             batch_size=config["validation_batch_size"],
                                             shuffle=False, 
                                             num_workers=8)

    attr_len = len(train_dataset[0][2]) #Number of attributes in the csv: 27
    print(f"Number of attributes: {attr_len}")
    model = FinetunedModel(architecture=config["backbone"] ,n_classes=attr_len).to(device)
    
    criterion = AttributesLossWrapper(2) #TODO: So far it optimizes only the attributes but this loss should be the combination of identification and attributes
    optimizer = torch.optim.SGD(model.parameters(), 
                                lr=config["learning_rate"], 
                                weight_decay=config["weight_decay"], 
                                momentum=config["momentum"])
    
    return model, train_loader, val_loader, val_queries_loader, criterion, optimizer

In [ ]:
def train(model, train_loader, val_loader, criterion, optimizer, config):
    # tell wandb to watch what the model gets up to: gradients, weights, and more!
    # if config["wandb"]:
    #     wandb.watch(model, criterion, log="all", log_freq=10)
    model.train()
    # Run training and track with wandb
    total_batches = len(train_loader) * config["epochs"]
    example_ct = 0  # number of seen examples
    batch_ct = 0

    for epoch in tqdm(range(config["epochs"])):
        for batch_idx, (inputs, identity, attributes) in enumerate(train_loader):
            loss, predicted = train_batch(inputs, identity, attributes, model, optimizer, criterion)

            example_ct +=  len(inputs)
            batch_ct += 1

            if ((batch_ct + 1) % 50) == 0:
                train_log(loss, example_ct, epoch)


def train_batch(inputs, identity, attributes, model, optimizer, criterion):
    inputs = inputs.to(device) #torch.Size([batch_size, 3, 224, 224])
    print('identity: ', identity)
    # print('identity size: ', identity.size())
    print('inputs: ', inputs)
    print('inputs size: ', inputs.size())

    # id = inputs.id.random -> len>1 (2 photos)
    # class tripletLoss(anchor,positive, negative)

    # anchor = id_1
    # positive = id_2
    # negative = diff_id.random

    # print('inputs.size(): ',inputs.size())
    # print('inputs: ',inputs)

    attributes = attributes.to(device)
    
    # Forward pass
    outputs = model(inputs)
    # Apply the loss
    loss = criterion(outputs, attributes)
    
    optimizer.zero_grad()
    # Backward pass
    loss.backward()

    # Step with optimizer
    optimizer.step()

    # Zeros the gradient
    predicted = torch.round(outputs[0].squeeze(1))

    quit()
    return loss, predicted

In [ ]:
train_set, val_set, val_queries = split_data(config)
model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
train(model, train_loader, val_loader, criterion, optimizer, config)


Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9705
Validation set size: 2922
Number of validation queries: 362
Number of attributes: 27


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Backbone feature size: 512


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


identity:  ('1064', '1076', '0968', '1247', '1174', '1423', '0090', '0607')
inputs:  tensor([[[[-1.2445, -1.2445, -1.2445,  ..., -1.1075, -1.1075, -1.1075],
          [-1.2274, -1.2274, -1.2274,  ..., -1.1075, -1.1075, -1.1075],
          [-1.2103, -1.2103, -1.2103,  ..., -1.1075, -1.1075, -1.1075],
          ...,
          [ 0.9646,  0.9646,  0.9646,  ...,  0.6734,  0.6563,  0.6563],
          [ 0.8104,  0.8104,  0.8104,  ...,  0.7077,  0.6906,  0.6906],
          [ 0.7248,  0.7248,  0.7248,  ...,  0.7248,  0.7077,  0.7077]],

         [[-1.0903, -1.0903, -1.0903,  ..., -0.9503, -0.9503, -0.9503],
          [-1.0728, -1.0728, -1.0728,  ..., -0.9503, -0.9503, -0.9503],
          [-1.0553, -1.0553, -1.0553,  ..., -0.9503, -0.9503, -0.9503],
          ...,
          [ 0.9230,  0.9230,  0.9230,  ...,  0.8004,  0.7829,  0.7829],
          [ 0.7654,  0.7654,  0.7654,  ...,  0.8354,  0.8179,  0.8179],
          [ 0.6779,  0.6779,  0.6779,  ...,  0.8529,  0.8354,  0.8354]],

         [[-0.706

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Streaming output truncated to the last 5000 lines.
          ...,
          [ 0.3652,  0.3652,  0.3652,  ...,  0.4337,  0.4337,  0.4337],
          [ 0.5878,  0.5878,  0.5878,  ...,  0.3652,  0.3652,  0.3652],
          [ 0.7419,  0.7419,  0.7419,  ...,  0.3138,  0.3138,  0.3138]],

         [[ 0.5378,  0.5378,  0.5378,  ...,  0.1176,  0.2227,  0.2227],
          [ 0.4153,  0.4153,  0.4153,  ...,  0.1352,  0.2052,  0.2052],
          [ 0.2227,  0.2227,  0.2052,  ...,  0.1877,  0.1702,  0.1702],
          ...,
          [ 0.4678,  0.4678,  0.4678,  ...,  0.5378,  0.5378,  0.5378],
          [ 0.6954,  0.6954,  0.6954,  ...,  0.4678,  0.4678,  0.4678],
          [ 0.8529,  0.8529,  0.8529,  ...,  0.4153,  0.4153,  0.4153]],

         [[ 0.0256,  0.0256,  0.0256,  ...,  0.0953,  0.1999,  0.1999],
          [-0.0790, -0.0790, -0.0790,  ...,  0.1302,  0.1999,  0.1999],
          [-0.2358, -0.2358, -0.2532,  ...,  0.1999,  0.1999,  0.1999],
          ...,
          [ 0.7054,  0.7054,  0.7054

NameError: ignored

In [ ]:
import torch
# embeddings = len(inputs) #batch function
# inputs size:  torch.Size([8, 3, 224, 224])

def pairwise_distances(embeddings, squared=False):
	"""
	||a-b||^2 = |a|^2 - 2*<a,b> + |b|^2

	"""
	# get dot product (batch_size, batch_size)
	dot_product = embeddings.mm(embeddings.t()) #what is '.mm'?  matrix multiplication of the matrices input and mat2. https://pytorch.org/docs/stable/generated/torch.mm.html

	# a vector
	square_sum = dot_product.diag() #'.diag()' get the diagonal elements of a vector.

	distances = square_sum.unsqueeze(1) - 2*dot_product + square_sum.unsqueeze(0)

	distances = distances.clamp(min=0) #remove 'negative' distance

	if not squared:
		epsilon=1e-16
		mask = torch.eq(distances, 0).float()
		distances += mask * epsilon
		distances = torch.sqrt(distances)
		distances *= (1-mask)

	return distances

def get_valid_positive_mask(labels):
	"""
	To be a valid positive pair (a,p),
		- a and p are different embeddings -> different image
		- a and p have the same label -> same person
	"""
	indices_equal = torch.eye(labels.size(0)).byte() #Returns a 2-D tensor with ones on the diagonal and zeros elsewhere.
	indices_not_equal = ~indices_equal #bitewise negation. Gonna take the opposite of the previous line.

	label_equal = torch.eq(labels.unsqueeze(1), labels.unsqueeze(0))

	mask = indices_not_equal & label_equal
	return mask

def get_valid_negative_mask(labels):
	"""
	To be a valid negative pair (a,n),
		- a and n are different embeddings
		- a and n have the different label
	"""
	indices_equal = torch.eye(labels.size(0)).byte()
	indices_not_equal = ~indices_equal

	label_not_equal = torch.ne(labels.unsqueeze(1), labels.unsqueeze(0))

	mask = indices_not_equal & label_not_equal
	return mask


def get_valid_triplets_mask(labels):
	"""
	To be valid, a triplet (a,p,n) has to satisfy:  
		- a,p,n are distinct embeddings
		- a and p have the same label, while a and n have different label
	"""
	indices_equal = torch.eye(labels.size(0)).byte()
	indices_not_equal = ~indices_equal
	i_ne_j = indices_not_equal.unsqueeze(2)
	i_ne_k = indices_not_equal.unsqueeze(1)
	j_ne_k = indices_not_equal.unsqueeze(0)
	distinct_indices = i_ne_j & i_ne_k & j_ne_k

	label_equal = torch.eq(labels.unsqueeze(1), labels.unsqueeze(0))
	i_eq_j = label_equal.unsqueeze(2)
	i_eq_k = label_equal.unsqueeze(1)
	i_ne_k = ~i_eq_k
	valid_labels = i_eq_j & i_ne_k

	mask = distinct_indices & valid_labels
	return mask

def batch_all_triplet_loss(labels, embeddings, margin, squared=False):
	"""
	get triplet loss for all valid triplets and average over those triplets whose loss is positive.
	"""

	distances = pairwise_distances(embeddings, squared=squared)

	anchor_positive_dist = distances.unsqueeze(2)
	anchor_negative_dist = distances.unsqueeze(1)
	triplet_loss = anchor_positive_dist - anchor_negative_dist + margin

	# get a 3D mask to filter out invalid triplets
	mask = get_valid_triplets_mask(labels)

	triplet_loss = triplet_loss * mask.float()
	triplet_loss.clamp_(min=0)

	# count the number of positive triplets
	epsilon = 1e-16
	num_positive_triplets = (triplet_loss > 0).float().sum()
	num_valid_triplets = mask.float().sum()
	fraction_positive_triplets = num_positive_triplets / (num_valid_triplets + epsilon)

	triplet_loss = triplet_loss.sum() / (num_positive_triplets + epsilon)

	return triplet_loss, fraction_positive_triplets

def batch_hard_triplet_loss(labels, embeddings, margin, squared=False):
	"""
	- compute distance matrix
	- for each anchor a0, find the (a0,p0) pair with greatest distance s.t. a0 and p0 have the same label
	- for each anchor a0, find the (a0,n0) pair with smallest distance s.t. a0 and n0 have different label
	- compute triplet loss for each triplet (a0, p0, n0), average them
	"""
	distances = pairwise_distances(embeddings, squared=squared)

	mask_positive = get_valid_positive_mask(labels)
	hardest_positive_dist = (distances * mask_positive.float()).max(dim=1)[0]

	mask_negative = get_valid_negative_mask(labels)
	max_negative_dist = distances.max(dim=1,keepdim=True)[0]
	distances = distances + max_negative_dist * (~mask_negative).float()
	hardest_negative_dist = distances.min(dim=1)[0]

	triplet_loss = (hardest_positive_dist - hardest_negative_dist + margin).clamp(min=0)
	triplet_loss = triplet_loss.mean()

	return triplet_loss


In [ ]:
embeddings = [8, 3, 224, 224]
pairwise_distances(embeddings)

AttributeError: ignored

In [ ]:
#id_ground_truth_dict = build_ground_truth(val_set, val_queries)

In [ ]:
def train_log(loss, example_ct, epoch):
    loss = float(loss)
    if config["wandb"]:
        wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Epoch {epoch}: Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

In [ ]:
def get_features_from_loader(model, loader):
    model.eval()
    with torch.no_grad():
        all_features = torch.zeros(len(loader.dataset), model.feature_size)
        for batch_idx, (inputs, ids, attr) in enumerate(tqdm(loader)):
                inputs = inputs.to(device)
                features = model(inputs, get_features=True)
                for in_batch, f in enumerate(features):
                    all_features[(batch_idx * loader.batch_size) + in_batch] = f
        return all_features


def test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    """
    This function returns the mAP performance of a given model 
    Params:
    model: model to be evaluated
    gallery: tensor that contains the feature representations of the target images in validation or test set
    queries: tensor that contains feature representations of the queries
    rank: top number of elements to retrieve

    Returns:
    mAP performance of the model
    """

    # Run the model on some test examples
    with torch.no_grad():
        
        # Compute the features for queries and gallery
        print("Computing gallery features...")
        gallery_features = get_features_from_loader(model, gallery_loader)
        print("Computing query features...")
        query_features = get_features_from_loader(model, queries_loader)
        
        # Build the cosine similarity matrix between the all the queries and all the elements in gallery
        print("Computing cosine similarities...")
        sims_matrix = torch.empty(query_features.size()[0], gallery_features.size()[0])
        for idx, q in enumerate(query_features):
            sims_matrix[idx] = F.cosine_similarity(q, gallery_features, dim=-1)
        
        print("Similarity matrix shape: " + str(sims_matrix.size()))
        sorted_index = torch.argsort(sims_matrix, dim=1, descending=True)
        top_k = sorted_index.narrow_copy(dim=1, start=0, length=config["mAP_rank"])

        #Build the dictionary to compute the mAP
        predictions_dict = {idx:  r for idx, r in enumerate(top_k.tolist())}
        mAP = Evaluator.evaluate_map(predictions_dict, ground_truth_dict)
        
        print(f"mAP: {mAP}")
        if config["wandb"]:
            wandb.log({"mAP": mAP})

In [ ]:
def get_attributes_from_loader(model, loader):
    model.eval()

    all_predictions = np.empty(shape=[0, 27], dtype=np.byte)
    all_attrs = np.empty(shape=[0, 27], dtype=np.byte)

    with torch.no_grad():
        for batch_idx, (inputs, ids, attr) in enumerate(tqdm(loader)):
                inputs = inputs.to(device)
                outputs = model(inputs, get_features=False)
                #print("attr:",attr)
                predictions = torch.empty(attr.size()[1], attr.size()[0])
                for attr_idx, output in enumerate(outputs):
                    if output.size()[1] == 1: #If the output is binary
                        pred = torch.round(torch.squeeze(output, 1))
                    else: #Otherwise it is multiclass
                        pred = torch.argmax(output, dim=1)
                    predictions[attr_idx] = pred

                predictions = torch.transpose(predictions, 0, 1).cpu().numpy()
                attr = attr.cpu().numpy()

                all_predictions = np.append(all_predictions, predictions, axis=0)
                #print("all_predictions shape: ", all_predictions.shape)
                all_attrs = np.append(all_attrs, attr, axis=0)
                #print("all_attrs shape: ", all_attrs.shape)
        return all_predictions, all_attrs

def test_attributes(model, loader, config):
    print("Computing attributes...")
    predictions, attr = get_attributes_from_loader(model, loader)
    print("pred shape: ", predictions.shape)
    print("attr shape: ", attr.shape)

    accuracy_list = []
    precision_list = []
    recall_list = []
    f1_score_list = []

    for i in range(0, predictions.shape[1]):
        y_true, y_pred = attr[:, i], predictions[:, i]
        accuracy_list.append(accuracy_score(y_true, y_pred))
        if i == 0: #If it is age
            precision_list.append(precision_score(y_true, y_pred, average='macro'))
            recall_list.append(recall_score(y_true, y_pred, average='macro'))
            f1_score_list.append(f1_score(y_true, y_pred, average='macro'))
        else:
            precision_list.append(precision_score(y_true, y_pred))
            recall_list.append(recall_score(y_true, y_pred))
            f1_score_list.append(f1_score(y_true, y_pred))

    average_acc = np.mean(accuracy_list)
    average_precision = np.mean(precision_list)
    average_recall = np.mean(recall_list)
    average_f1score = np.mean(f1_score_list)

    print("accuracy_list: ", accuracy_list)
    print("precision_list: ", precision_list)
    print("recall_list: ", recall_list)
    print("f1_score_list: ", f1_score_list)

    print("average_acc: ", average_acc)
    print("average_precision: ", average_precision)
    print("average_recall: ", average_recall)
    print("average_f1score: ", average_f1score)

    if config["wandb"]:
            wandb.log({"accuracy_list": accuracy_list})
            wandb.log({"precision_list": precision_list})
            wandb.log({"recall_list": recall_list})
            wandb.log({"f1_score_list": f1_score_list})

            wandb.log({"average accuracy": average_acc})
            wandb.log({"average precision": average_precision})
            wandb.log({"average recall": average_recall})
            wandb.log({"average f1": average_f1score})

In [ ]:
def test(model, gallery_loader, queries_loader, ground_truth_dict, config, save_model=False):
    print("Testing")
    model.eval()

    test_mAP(model, gallery_loader, queries_loader, ground_truth_dict, config)
    test_attributes(model, gallery_loader, config)

    if save_model :
      # Save the model in the exchangeable ONNX format
      inputs, id, attr = next(iter(gallery_loader))
      torch.onnx.export(model, inputs.to(device), "model.onnx", True)
      if config["wandb"]:
        wandb.save("model.onnx")

In [ ]:
model = model_pipeline(config)

wandb: Currently logged in as: sango (use `wandb login --relogin` to force relogin)


{'wandb': True, 'device': 'auto', 'train_root': '/content/dataset/train', 'test_root': '/content/dataset/test', 'queries_root': '/content/dataset/queries', 'attributes_file': '/content/dataset/annotations_train.csv', 'dataset': 'Market1501', 'backbone': 'resnet18', 'split': {'full_training_size': 0.75, 'train_size': 0.8}, 'compose': {'resize_h': 224, 'resize_w': 224}, 'epochs': 1, 'training_batch_size': 8, 'validation_batch_size': 32, 'learning_rate': 0.01, 'weight_decay': 1e-06, 'momentum': 0.9, 'test_before_training': True, 'test_after_epochs': 10, 'mAP_rank': 10}
Extract queries proportion: 0.11
Identities in train set: 563
Identities in validation set: 188
Train set size: 9779
Validation set size: 2856
Number of validation queries: 354


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Number of attributes: 27


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


...

Epoch 0: Loss after 00392 examples: 12.128
Epoch 0: Loss after 00792 examples: 13.557
Epoch 0: Loss after 01192 examples: 10.024
Epoch 0: Loss after 01592 examples: 10.551
Epoch 0: Loss after 01992 examples: 9.116
Epoch 0: Loss after 02392 examples: 12.282
Epoch 0: Loss after 02792 examples: 12.138
Epoch 0: Loss after 03192 examples: 7.718
Epoch 0: Loss after 03592 examples: 8.396
Epoch 0: Loss after 03992 examples: 9.660
Epoch 0: Loss after 04392 examples: 9.685
Epoch 0: Loss after 04792 examples: 10.010
Epoch 0: Loss after 05192 examples: 8.629
Epoch 0: Loss after 05592 examples: 10.012
Epoch 0: Loss after 05992 examples: 9.005
Epoch 0: Loss after 06392 examples: 7.538
Epoch 0: Loss after 06792 examples: 8.623
Epoch 0: Loss after 07192 examples: 7.469
Epoch 0: Loss after 07592 examples: 7.651
Epoch 0: Loss after 07992 examples: 6.840
Epoch 0: Loss after 08392 examples: 9.253
Epoch 0: Loss after 08792 examples: 9.243
Epoch 0: Loss after 09192 examples: 8.144
Epoch 0: Loss after 09592 


Computing query features...



Computing cosine similarities...
Similarity matrix shape: torch.Size([354, 2856])
mAP: 0.13050297052308127
Computing attributes...



pred shape:  (2856, 27)
attr shape:  (2856, 27)
accuracy_list:  [0.8490896358543417, 0.6750700280112045, 0.7531512605042017, 0.9257703081232493, 0.8319327731092437, 0.8245798319327731, 0.9394257703081232, 0.6929271708683473, 0.9989495798319328, 0.7181372549019608, 0.9359243697478992, 0.8105742296918768, 0.9061624649859944, 0.9754901960784313, 0.9572829131652661, 0.8778011204481793, 0.9198179271708683, 0.9215686274509803, 0.7717086834733894, 0.9334733893557423, 0.9600840336134454, 1.0, 1.0, 0.8126750700280112, 0.7643557422969187, 0.969187675070028, 0.9747899159663865]
precision_list:  [0.21227240896358543, 0.19548872180451127, 0.0, 0.0, 0.8406533575317604, 0.9229442208165612, 0.9394257703081232, 0.7279693486590039, 0.0, 0.7681912681912682, 0.7181208053691275, 0.6780551905387647, 0.5219638242894057, 0.0, 0.9601769911504425, 0.4383561643835616, 0.5384615384615384, 0.6988636363636364, 0.9022556390977443, 0.6923076923076923, 0.0, 0.0, 0.0, 0.43333333333333335, 0.4493975903614458, 0.0, 0.72

/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1515: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  average, "true nor predicted", 'F-score is', len(true_sum)


In [ ]:
#train_set, val_set, val_queries = split_data(config)
#model, train_loader, val_loader, val_queries_loader, criterion, optimizer = setup(train_set, val_set, val_queries, config)
#id_ground_truth_dict = build_ground_truth(val_set, val_queries)